In [46]:
import pandas as pd
df = pd.read_csv('/kaggle/input/bhw1-data/bhw1/labels.csv')
df.head()

,Id,Category
0,trainval_00000.jpg,7
1,trainval_00001.jpg,198
2,trainval_00002.jpg,161
3,trainval_00003.jpg,131
4,trainval_00004.jpg,107


Number of categories:

In [47]:
df['Category'].nunique()

200

Dataset and Dataloader

In [48]:
import os
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms

class CustomImageDataset(Dataset):
    def __init__(self, annotations_file, img_dir, transform=None, target_transform=None):
        self.img_labels = annotations_file
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        #print(idx)
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
        image = Image.open(img_path).convert("RGB")
        label = int(self.img_labels.iloc[idx, 1])

        if self.transform:
            image = self.transform(image)

        if self.target_transform:
            label = self.target_transform(label)

        return image, label

In [49]:
class TestImageDataset(Dataset):
    def __init__(self, img_dir, transform=None):
        
        self.img_dir = img_dir
        
        self.img_names = os.listdir(img_dir)
        
        
        self.transform = transform

    def __len__(self):
        return len(self.img_names)

    
    
    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_names[idx])
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)

        return image, self.img_names[idx]

In [50]:
def classify(model, test_dataset_path):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    test_dataset = TestImageDataset(test_dataset_path, transform=transforms.Compose([transforms.Resize([40, 40]), transforms.ToTensor(), 
                                                                                     transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                                                                        std=[0.229, 0.224, 0.225])]))
    dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)      
    predictions = []
    file_names = []

    
    for inputs, file_name in tqdm(dataloader):
        inputs = inputs.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        predictions.extend(predicted.tolist())
        file_names.extend(file_name)

    predictions_with_filenames = dict(zip(file_names, predictions))
    

    return predictions_with_filenames, predictions

In [51]:
dataset = CustomImageDataset(df,"bhw1/trainval/" , transform=transforms.Compose([transforms.Resize(40), transforms.ToTensor()]))
dataloader = DataLoader(dataset, batch_size=256, shuffle=True)

Model:

In [52]:
import os
from PIL import Image
from tqdm.notebook import tqdm
import torch
import torchvision.transforms as T
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import torch.optim.lr_scheduler as lr_scheduler

import torch
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader

In [55]:
class MyModel(nn.Module):
    def __init__(self, num_classes):
        super(MyModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, padding=1, bias=True)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(64, 64, kernel_size=3, padding=1, bias=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        original_resnet = models.resnet18(num_classes=200)
        
        self.features = nn.Sequential(*list(original_resnet.children())[4:-1])
        self.bn2 = nn.BatchNorm1d(512)
        self.fc1 = nn.Linear(512, 512)
        self.fc2 = nn.Linear(512, 200)

        for param in self.parameters():
            param.requires_grad = True


    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.conv2(x)

        x = self.relu(x)
        x = self.features(x)
        x = self.fc1(x)
        x = self.bn2(x)
        x = torch.relu(x)
        x = self.fc2(x)


        return x

Training:

In [73]:
from torch.utils.data import random_split
def train_classifier(df, train_img_dir, fast_train=False):

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    dataset = CustomImageDataset(df,train_img_dir , transform=transforms.Compose([transforms.RandomResizedCrop(40, scale = (0.8, 1.1)),
                                                                                  transforms.RandomHorizontalFlip(),
                                                                                  transforms.RandomAdjustSharpness(2, p=0.5),
                                                                                  transforms.RandomRotation((-10, 10)),
                                                                                  transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.1),
                                                                                  #transforms.RandomErasing(p=0.1, scale=(0.02, 0.10), ratio=(0.3, 3.3), value='random'),
                                                                                  
                                                                                  #transforms.Resize([40, 40]),
                                                                                  transforms.ToTensor(),
                                                                                  transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                                                                       std=[0.229, 0.224, 0.225])]))
    #train_size = int(0.98 * len(dataset))
    #test_size = len(dataset) - train_size
    #dataset, val_dataset = random_split(dataset, [train_size, test_size])
    dataloader = DataLoader(dataset, batch_size=256, shuffle=True)
    #val_dataloader = DataLoader(val_dataset, batch_size=256, shuffle=True)
    
    model = MyModel(num_classes = 200).to(device)
    #model = FourthModel().to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    #milestones = [5]
    #gamma = 0.1
    #scheduler = lr_scheduler.MultiStepLR(optimizer, milestones, gamma)
    for epoch in range(30):
        print(epoch)
        s = 0
        tot = 0
        #model.train()
        for inputs, labels in tqdm(dataloader):
            #print(inputs.shape)
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs).to(device)

            loss = criterion(outputs, labels)

            optimizer.zero_grad()

            loss.backward()


            optimizer.step()
        #model.eval()
#         for inputs, labels in tqdm(val_dataloader):
#             inputs, labels = inputs.to(device), labels.to(device)
#             outputs = model(inputs).to(device)
#             _, predicted = torch.max(outputs, 1)
#             correct = (predicted == labels).sum().item()
#             s += correct
#             tot += labels.size(0)
#         print("Accuracy: ", s / tot)
        
        print(f'Epoch [{epoch+1}/{30}], Loss: {loss.item()}')
    return model

In [74]:
model = train_classifier(df, "/kaggle/input/bhw1-data/bhw1/trainval/")

0


  0%|          | 0/391 [00:00<?, ?it/s]

Epoch [1/30], Loss: 4.479461669921875
1


  0%|          | 0/391 [00:00<?, ?it/s]

Epoch [2/30], Loss: 3.867403745651245
2


...

Epoch [28/30], Loss: 1.2987210750579834
28


  0%|          | 0/391 [00:00<?, ?it/s]

Epoch [29/30], Loss: 0.9033647775650024
29


  0%|          | 0/391 [00:00<?, ?it/s]

Epoch [30/30], Loss: 1.0330408811569214


Testing:

In [75]:
class TestImageDataset(Dataset):
    def __init__(self, img_dir, transform=None):
        
        self.img_dir = img_dir
        
        self.img_names = os.listdir(img_dir)
        
        
        self.transform = transform

    def __len__(self):
        return len(self.img_names)

    
    
    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_names[idx])
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)

        return image, self.img_names[idx]

In [76]:
def classify(model, test_dataset_path):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    test_dataset = TestImageDataset(test_dataset_path, transform=transforms.Compose([transforms.Resize([40, 40]), transforms.ToTensor(), 
                                                                                     transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                                                                        std=[0.229, 0.224, 0.225])]))
    dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)      
    predictions = []
    file_names = []

    
    for inputs, file_name in tqdm(dataloader):
        inputs = inputs.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        predictions.extend(predicted.tolist())
        file_names.extend(file_name)

    predictions_with_filenames = dict(zip(file_names, predictions))
    

    return predictions_with_filenames, predictions

In [77]:
predicts, preds = classify(model, '/kaggle/input/bhw1-data/bhw1/test/')


  0%|          | 0/313 [00:00<?, ?it/s]

In [78]:
df_test = pd.read_csv('/kaggle/input/bhw1-data/bhw1/sample_submission.csv')
df_test.head()

,Id,Category
0,test_00000.jpg,153
1,test_00001.jpg,70
2,test_00002.jpg,194
3,test_00003.jpg,171
4,test_00004.jpg,38


In [80]:
import csv
predicts = predicts
csv_filename = '/kaggle/working/submission.csv'

# Записываем словарь в CSV файл
with open(csv_filename, 'w', newline='') as csv_file:
    writer = csv.writer(csv_file)
    # Записываем заголовок
    writer.writerow(['Id', 'Category'])
    # Записываем значения из словаря
    for key, value in predicts.items():
        writer.writerow([key, value])
        #print([key, value])
print(f"CSV file {csv_filename} sucessfully created.")

CSV файл /kaggle/working/submission.csv успешно создан.
